<a href="https://colab.research.google.com/github/patelruday/GenerativeAI/blob/main/pdf_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"OM NAMO BHAGVATE VASUDEVAY"

'OM NAMO BHAGVATE VASUDEVAY'

In [ ]:
import streamlit as st
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
import uuid
import PyPDF2
import google.generativeai as genai
import faiss
import tempfile
from langchain.docstore.in_memory import InMemoryDocstore
from langchain_groq import ChatGroq
# Initialize embeddings, FAISS, and LLM
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Set up the FAISS index (L2 similarity)
dimension = 384  # Dimension for the 'all-MiniLM-L6-v2' embeddings
index = faiss.IndexFlatL2(dimension)

# Set up an in-memory docstore to keep track of documents
docstore = InMemoryDocstore({})
index_to_docstore_id = {}

# Initialize the FAISS vector store
faiss_index = FAISS(
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id,
    embedding_function=embedding_model.embed_query
)

# Configure the LLM
llm=ChatGroq(
    model="llama-3.1-70b-versatile",
    groq_api_key="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",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# Function to load and split PDFs
def load_and_split_pdf(file_path):
    pdf_reader = PyPDF2.PdfReader(file_path)
    text_chunks = []
    for page_num in range(len(pdf_reader.pages)):
        page_text = pdf_reader.pages[page_num].extract_text()
        # Split text into smaller chunks for processing
        if page_text:
            chunks = [page_text[i:i + 500] for i in range(0, len(page_text), 500)]
            text_chunks.extend(chunks)
    return text_chunks

# Function to embed documents
# Function to embed documents
def embed_documents(text_chunks):
    for chunk in text_chunks:
        # Generate a unique ID for each document
        doc_id = str(uuid.uuid4())
        # Add the chunk (text) to the FAISS index, including metadata as a dictionary
        faiss_index.add_texts([chunk], [{"doc_id": doc_id}])

# Function to handle chat queries
def chat_with_pdfs(query):
    query_embedding = embedding_model.embed_query(query)
    results = faiss_index.similarity_search_by_vector(query_embedding, k=5)
    context = " ".join([result.page_content for result in results])
    response = llm.invoke(context + "\n" + query)
    return response

# Streamlit UI
st.title("Chat with Your PDF Documents")
st.write("Upload multiple PDF documents and chat with them!")

uploaded_files = st.file_uploader("Upload PDFs", type="pdf", accept_multiple_files=True)

if uploaded_files:
    st.write("Processing PDFs...")
    for uploaded_file in uploaded_files:
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            temp_file.write(uploaded_file.read())
            text_chunks = load_and_split_pdf(temp_file.name)
            embed_documents(text_chunks)
    st.write("PDFs have been successfully processed. You can now ask questions!")

# Chat interface
query = st.text_input("Ask a question about your PDFs:")
if query:
    response = chat_with_pdfs(query)
    st.write("Answer:", response.content)
